In [ ]:
import os
from PIL import Image
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TinyImageNetDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.image_paths, self.labels = self._load_data()

    def _load_data(self):
        image_paths = []
        labels = []

        if self.split == 'train_':
            # Load train data
            train_dir = os.path.join(self.root_dir,train)
            class_folders = os.listdir(train_dir)
            for class_folder in class_folders:
                class_path = os.path.join(train_dir, class_folder)
                image_files = os.listdir(class_path)
                image_files = [path for path in image_files if not path.endswith('.txt')]
                for image_file in image_files:
                    image_paths.append(os.path.join(class_path, image_file))
                    labels.append(class_folder)

        elif self.split == 'test_':
            # Load test data
            test_dir = os.path.join(self.root_dir,'test','images')
            image_files = os.listdir(test_dir)
            for image_file in image_files:
                image_paths.append(image_file)
                labels.append(None)  

        elif self.split == 'val_':
            # Load validation data
            val_dir = os.path.join(self.root_dir,'val')

            image_dir = os.path.join(val_dir,'images')
            img_files = os.listdir(image_dir)
            for image_file in img_files:
                image_paths.append(image_file)
            
            annotation_file = os.path.join(val_dir,'val_annotations.txt')
            annotations_df = pd.read_csv(annotation_file, sep='\t', header=None)
            for _, row in annotations_df.iterrows():
                image_paths.append(os.path.join(val_dir, row[0]))
                labels.append(row[1])

        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.split, self.image_paths[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = TinyImageNetDataset(root_dir='F:\\Research\\tiny-imagenet-200', split='train_', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TinyImageNetDataset(root_dir='F:\\Research\\tiny-imagenet-200', split='test_', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

val_dataset = TinyImageNetDataset(root_dir='F:\\Research\\tiny-imagenet-200', split='val_', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [ ]:
import torch.nn as nn

class TinyNet(nn.Module):
    def __init__(self):
        super(TinyNet, self).__init__()
        self.fc1 = nn.Linear(3 * 224 * 224, 128)
        self.fc2 = nn.Linear(128, 200)  # 10 classes for CIFAR-10

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = TinyNet()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_one_epoch():
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


In [ ]:
for epoch in range(100):
    train_one_epoch()

In [ ]:
import matplotlib.pyplot as plt

def validate(model, val_loader):
    model.eval()  
    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_samples += len(labels)

    average_loss = total_loss / total_samples
    return average_loss


def plot_losses(train_losses, test_losses):
    plt.plot(range(len(train_losses)), train_losses, label='Train Loss')
    plt.plot(range(len(test_losses)), test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


train_losses, test_losses = [], []
for epoch in range(100):
    train_loss = train_one_epoch()  
    test_loss = validate(model, val_loader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)

plot_losses(train_losses, test_losses)